In [1]:
import pandas as pd
import re
import string
import nltk
import seaborn as sns
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [2]:
#Membuka Data Frame

data = pd.read_csv('data.csv')
kamus_alay = pd.read_csv('new_kamusalay.csv', encoding='latin1', header =None)

In [3]:
data.shape #terdapat 13 kolom dan 13169 data

(13169, 13)

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13169 entries, 0 to 13168
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Tweet          13169 non-null  object
 1   HS             13169 non-null  int64 
 2   Abusive        13169 non-null  int64 
 3   HS_Individual  13169 non-null  int64 
 4   HS_Group       13169 non-null  int64 
 5   HS_Religion    13169 non-null  int64 
 6   HS_Race        13169 non-null  int64 
 7   HS_Physical    13169 non-null  int64 
 8   HS_Gender      13169 non-null  int64 
 9   HS_Other       13169 non-null  int64 
 10  HS_Weak        13169 non-null  int64 
 11  HS_Moderate    13169 non-null  int64 
 12  HS_Strong      13169 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 1.3+ MB


In [5]:
#Daftar kolom yang terdapat pada Data

data.columns

Index(['Tweet', 'HS', 'Abusive', 'HS_Individual', 'HS_Group', 'HS_Religion',
       'HS_Race', 'HS_Physical', 'HS_Gender', 'HS_Other', 'HS_Weak',
       'HS_Moderate', 'HS_Strong'],
      dtype='object')

In [6]:
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0


***MISSING VALUE***

In [7]:
data.isna().sum()

Tweet            0
HS               0
Abusive          0
HS_Individual    0
HS_Group         0
HS_Religion      0
HS_Race          0
HS_Physical      0
HS_Gender        0
HS_Other         0
HS_Weak          0
HS_Moderate      0
HS_Strong        0
dtype: int64

***Catatan*** = Tidak ada missing value

***DUPLICATED DATA***

In [8]:
#Mencari data yang duplicate

data.loc[data.duplicated(keep=False)].sort_values('Tweet')

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong
43,#GubernurZamanNow #GusIpulPuti2 #GanjarYasin1 ...,0,0,0,0,0,0,0,0,0,0,0,0
10254,#GubernurZamanNow #GusIpulPuti2 #GanjarYasin1 ...,0,0,0,0,0,0,0,0,0,0,0,0
378,#GubernurZamanNow #GusIpulPuti2 #GanjarYasin1 ...,0,0,0,0,0,0,0,0,0,0,0,0
4254,#GubernurZamanNow #GusIpulPuti2 #GanjarYasin1 ...,0,0,0,0,0,0,0,0,0,0,0,0
295,#GubernurZamanNow #GusIpulPuti2 #GanjarYasin1 ...,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1768,masuknya Jokowi sebagai tokoh muslim yang berp...,0,0,0,0,0,0,0,0,0,0,0,0
8544,tangkap ahok dan klompok nya yg melakukan pema...,1,0,0,1,0,0,0,0,1,0,0,1
5182,tangkap ahok dan klompok nya yg melakukan pema...,1,0,0,1,0,0,0,0,1,0,0,1
11964,"~; Yg jelas, klw ada foto2 yg begitu:; #lombaT...",0,0,0,0,0,0,0,0,0,0,0,0


In [9]:
data = data.drop_duplicates() #Menghilangkan Duplicated Data

In [10]:
data.duplicated().sum() #Memeriksa Duplicated Data

0

***MENCARI TAU JUMLAH KARAKTER & KATA SETIAP ROW PADA KOLOM TWEET***

In [11]:
data['total_char'] = data['Tweet'].apply(len)

In [12]:
data['total_word'] = data['Tweet'].apply(lambda sent: len(sent.split()))

In [13]:
data.head()

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,total_char,total_word
0,- disaat semua cowok berusaha melacak perhatia...,1,1,1,0,0,0,0,0,1,1,0,0,138,25
1,RT USER: USER siapa yang telat ngasih tau elu?...,0,1,0,0,0,0,0,0,0,0,0,0,120,21
2,"41. Kadang aku berfikir, kenapa aku tetap perc...",0,0,0,0,0,0,0,0,0,0,0,0,254,37
3,USER USER AKU ITU AKU\n\nKU TAU MATAMU SIPIT T...,0,0,0,0,0,0,0,0,0,0,0,0,75,14
4,USER USER Kaum cebong kapir udah keliatan dong...,1,1,0,1,1,0,0,0,0,0,1,0,89,14


***CLEANSING DATA***

In [44]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [45]:
#Mengubah semua huruf menjadi huruf kecil
def lowercase(text):
    lower_text = text.lower()
    return lower_text

#Menghilangkan kata user
def remove_user(text):
    text = re.sub('user',' ', text)
    text = re.sub(' +',' ', text)
    return text

#Clean text
def clean_text(text):
    text = re.sub('[0-9]+', ' ', text) #remove angka
    text = re.sub(r'^RT[\s]+', ' ', text) #remove RT
    text = re.sub(r'^rt[\s]+', ' ', text) #remove rt
    text = re.sub('\n',' ', text) #menghilangkan new line
    text = re.sub(r'#', ' ', text) #remove hashtags
    text = re.sub('[^0-9a-zA-Z]',' ', text) #menghilangkan angka yang bercampur huruf
    text = text.strip() #menghapus whitespace/karakter kosong
    text = re.sub(' +',' ', text) #menghilangkan ekstra spasi
    text = re.sub(r'[^\w\s]',' ',text) #menghilangkan tanda baca
    return text

def remove_emoticon(text):
    text = text.replace(' \ ', ' ')
    text = re.sub('x..', ' ', text)
    text = re.sub(' n ', ' ', text)
    text = re.sub('\+', ' ', text)
    text = re.sub(' +',' ', text)
    text = re.sub(r'\$\w*',' ',text)
    return text

def stemming(text):
    return stemmer.stem(text)

def cleansing(text):
    text = lowercase(text)
    text = remove_user(text)
    text = clean_text(text)
    text = remove_emoticon(text)
    text = stemming(text)
    return text

In [46]:
for text in data['Tweet']:
    print(cleansing(text))

saat semua cowok usaha lacak perhati gue loe lantas remeh perhati yg gue kasih khusus ke elo basic elo cowok bego
siapa yang telat ngasih tau elu edan sarap gue gaul dengan ciga ifla cal sama siapa noh licew juga
kadang aku berfikir kenapa aku tetap percaya pada tuhan padahal aku selalu jatuh kali kali kadang aku rasa tuhan itu ninggalkan aku sendiri ketika orangtuaku rencana pisah ketika kakak lebih pilih jadi kristen ketika aku anak ter
aku itu aku nku tau mata sipit tapi liat dari mana itu aku
kaum cebong kapir udah liat dongok dari awal tambah dongok lagi hahahah
ya bani taplak dkk
deklarasi pilkada aman dan anti hoa arga dukuh sari jabon
gue baru aja kelar re watch aldnoah zero paling kampret emang endingnya karakter utama cowonya kena friendzone bray url
nah admin belanja satu lagi port baik nak makan ais kepal milo ais kepal horlicks atau cendol toping kaw kaw doket mano tu gerai rojak mertua taipan depan twins baby amp romantika bank islam senawang
enak lg klo smbil ngewe
tidak

In [48]:
data['Tweet'] = data['Tweet'].apply(cleansing)

In [51]:
data.head(6)

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,total_char,total_word
0,saat semua cowok usaha lacak perhati gue loe l...,1,1,1,0,0,0,0,0,1,1,0,0,138,25
1,siapa yang telat ngasih tau elu edan sarap gue...,0,1,0,0,0,0,0,0,0,0,0,0,120,21
2,kadang aku berfikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0,254,37
3,aku itu aku nku tau mata sipit tapi liat dari ...,0,0,0,0,0,0,0,0,0,0,0,0,75,14
4,kaum cebong kapir udah liat dongok dari awal t...,1,1,0,1,1,0,0,0,0,0,1,0,89,14
5,ya bani taplak dkk,1,1,0,1,0,0,0,0,1,0,1,0,73,6


In [52]:
data.tail(7)

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,total_char,total_word
13162,kapolda babi biadap dan bodoh gak punya otak kali,1,1,0,1,0,0,0,0,1,0,1,0,52,9
13163,orang yahudi kristen dan muslim yang belum per...,0,0,0,0,0,0,0,0,0,0,0,0,236,33
13164,jangan asal ngomong ndasmu congor lu yg kate a...,1,1,1,0,0,0,1,0,0,1,0,0,61,10
13165,kasur mana enak kunyuk,0,1,0,0,0,0,0,0,0,0,0,0,28,5
13166,hati hati bisu g nlagi bosan huft aa,0,0,0,0,0,0,0,0,0,0,0,0,62,9
13167,bom yang real mudah deteksi bom yang kubur sua...,0,0,0,0,0,0,0,0,0,0,0,0,135,22
13168,mana situ ngasih itu cuma foto ya kutil onta,1,1,1,0,0,0,0,0,1,1,0,0,53,10


**KAMUS ALAY**

In [55]:
kamus_alay.head()

,0,1
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [56]:
kamus_alay = kamus_alay.rename(columns={
    0:'original',
    1:'replace'
})

In [57]:
kamus_alay.head()

,original,replace
0,anakjakartaasikasik,anak jakarta asyik asyik
1,pakcikdahtua,pak cik sudah tua
2,pakcikmudalagi,pak cik muda lagi
3,t3tapjokowi,tetap jokowi
4,3x,tiga kali


In [58]:
alay_dict = dict(zip(kamus_alay['original'], kamus_alay['replace']))

In [59]:
def remove_kata_alay(text):
    return ' '.join([alay_dict[word] if word in alay_dict else word for word in text.split(' ')])

In [60]:
for text in data['Tweet']:
    print(remove_kata_alay(text))

saat semua cowok usaha lacak perhati gue kamu lantas remeh perhati yang gue kasih khusus ke kamu basic kamu cowok bego
siapa yang telat memberi tau kamu edan sarap gue gaul dengan ciga ifla cal sama siapa itu licew juga
kadang aku berpikir kenapa aku tetap percaya pada tuhan padahal aku selalu jatuh kali kali kadang aku rasa tuhan itu meninggalkan aku sendiri ketika orang tuaku rencana pisah ketika kakak lebih pilih jadi kristen ketika aku anak ter
aku itu aku ku tau mata sipit tapi lihat dari mana itu aku
kaum cebong kafir sudah lihat dungu dari awal tambah dungu lagi haha
ya bani taplak dan kawan kawan
deklarasi pilihan kepala daerah aman dan anti hoa arga dukuh sari jabon
gue baru saja selesai re watch aldnoah zero paling kampret memang akhirnya karakter utama cowoknya kena friendzone bro uniform resource locator
nah admin belanja satu lagi port baik nak makan ais kepal milo ais kepal horlicks atau cendol toping kau kau doket mana itu gerai rozak mertua taipan depan kembar baby amp 

In [61]:
data.head(8)

,Tweet,HS,Abusive,HS_Individual,HS_Group,HS_Religion,HS_Race,HS_Physical,HS_Gender,HS_Other,HS_Weak,HS_Moderate,HS_Strong,total_char,total_word
0,saat semua cowok usaha lacak perhati gue loe l...,1,1,1,0,0,0,0,0,1,1,0,0,138,25
1,siapa yang telat ngasih tau elu edan sarap gue...,0,1,0,0,0,0,0,0,0,0,0,0,120,21
2,kadang aku berfikir kenapa aku tetap percaya p...,0,0,0,0,0,0,0,0,0,0,0,0,254,37
3,aku itu aku nku tau mata sipit tapi liat dari ...,0,0,0,0,0,0,0,0,0,0,0,0,75,14
4,kaum cebong kapir udah liat dongok dari awal t...,1,1,0,1,1,0,0,0,0,0,1,0,89,14
5,ya bani taplak dkk,1,1,0,1,0,0,0,0,1,0,1,0,73,6
6,deklarasi pilkada aman dan anti hoa arga dukuh...,0,0,0,0,0,0,0,0,0,0,0,0,64,11
7,gue baru aja kelar re watch aldnoah zero palin...,0,1,0,0,0,0,0,0,0,0,0,0,129,20
